## Google Sheet Python 

https://developers.google.com/sheets/api/quickstart/python?hl=fr

pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [1]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [6]:
# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/spreadsheets.readonly"]

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = "1ir3Kk66cKF_SaK25Yj1A_w60gJlLasYDT1NQlQRGOVI"
SAMPLE_RANGE_NAME = "Class Data!A2:E"

In [8]:
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            "credentials.json", SCOPES
        )
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
        token.write(creds.to_json())
try:
    service = build("sheets", "v4", credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result = (
        sheet.values()
        .get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME)
        .execute()
    )
    values = result.get("values", [])

    if not values:
        print("No data found.")

    print("Name, Major:")
    for row in values:
        # Print columns A and E, which correspond to indices 0 and 4.
        print(f"{row[0]}, {row[4]}")
except HttpError as err:
    print(err)

<HttpError 400 when requesting https://sheets.googleapis.com/v4/spreadsheets/1ir3Kk66cKF_SaK25Yj1A_w60gJlLasYDT1NQlQRGOVI/values/Class%20Data%21A2%3AE?alt=json returned "Unable to parse range: Class Data!A2:E". Details: "Unable to parse range: Class Data!A2:E">


## Request

In [16]:
import re
import pandas as pd
from dotenv import load_dotenv

In [17]:
load_dotenv()

True

In [18]:
def convert_google_sheet_url(url):
    # Regular expression to match and capture the necessary part of the URL
    pattern = r'https://docs\.google\.com/spreadsheets/d/([a-zA-Z0-9-_]+)(/edit#gid=(\d+)|/edit.*)?'

    # Replace function to construct the new URL for CSV export
    # If gid is present in the URL, it includes it in the export URL, otherwise, it's omitted
    replacement = lambda m: f'https://docs.google.com/spreadsheets/d/{m.group(1)}/export?' + (f'gid={m.group(3)}&' if m.group(3) else '') + 'format=csv'

    # Replace using regex
    new_url = re.sub(pattern, replacement, url)

    return new_url

In [19]:
url = os.getenv("GOOGLE_SHEET_URL")

new_url = convert_google_sheet_url(url)

print(new_url)

https://docs.google.com/spreadsheets/d/1ir3Kk66cKF_SaK25Yj1A_w60gJlLasYDT1NQlQRGOVI/export?gid=530728187&format=csv


In [20]:
df = pd.read_csv(new_url)

In [21]:
df

,Horodateur,temperature,pressure,humidity,dewPoint,altitude,gas,moisture,counter
0,17/01/2024 11:36:29,27,979,51,16,286,324,394,1
1,17/01/2024 11:36:42,27,979,51,16,286,322,407,2
2,17/01/2024 11:37:37,27,979,51,16,286,320,407,5
3,17/01/2024 11:38:46,27,979,50,16,286,336,407,9
4,17/01/2024 11:39:55,27,979,50,16,286,350,407,13
5,17/01/2024 11:41:04,27,979,50,16,286,360,406,17
6,17/01/2024 11:42:26,27,979,50,15,286,384,407,22
7,17/01/2024 11:43:56,27,979,50,15,286,393,408,27
8,17/01/2024 12:30:18,21,977,41,7,303,306,407,2
9,17/01/2024 12:31:13,21,977,40,7,303,401,402,5
